In [1]:
import os
import numpy as np
import segyio

# Data to use

In [2]:
# Seismic data: SEG-Y format
near_stack = os.path.join("..", "data", "NS2950-2180_3000-2230.sgy")
mid_stack = os.path.join("..", "data", "MS2950-2180_3000-2230.sgy")
far_stack = os.path.join("..", "data", "FS2950-2180_3000-2230.sgy")

# To store G and I
g_i = os.path.join("..", "data", "GradInter.sgy")
# Wells data: txt format
wells_path = '../data/wells_info.txt'

# Store the seismic data in a list
gather_path = [near_stack, mid_stack, far_stack]

# Input parameters to work with


In [3]:
# Name of the survery to display in the results
seismic_survey = 'POSEIDON 3D'

# Name and path of the merged SEG-Y file
merged_stacks = os.path.join("..", "data", "NMF2950-2180_3000-2230.sgy")

# Gradient file (SEGY)
gradient_segy = os.path.join("..", "data", "gradient.sgy")
# Intercept file (SEGY)
intercept_segy = os.path.join("..", "data", "intercept.sgy")

# Angle between the gathers given
angle_bet_gathers = 12

# Making a list with the angle of the gathers
angle_list = []
angle = angle_bet_gathers
for file in range(len(gather_path)):
    angle_list = angle_list + [angle]
    angle = angle + angle_bet_gathers

# Merging the partial angle stacks


In [14]:
def merging_stacks(list_of_gathers, merged_file_path, list_of_angles):
    
    """
    
    source1: https://github.com/equinor/segyio/blob/master/python/examples/make-file.py
    source2: https://github.com/equinor/segyio/blob/master/python/segyio/tracefield.py
    
    """
    
    # Validating the files
#    for file in list_of_gathers:
#        if file_validation(file) == True:
#            print(f"{file} has been validated")
#        else:
#            return(f"Unsuccessful validation: {file_validation(file)}")
    
    # Making the offset array
    offsts = np.array(list_of_angles)

    # Initializing the stacks
    with segyio.open(list_of_gathers[0]) as f, segyio.open(list_of_gathers[1]) as g, segyio.open(list_of_gathers[2]) as h:
        
        # Spec function to build the new segy
        spec = segyio.spec()
        spec.sorting = f.sorting
        spec.format = 1
        spec.samples = f.samples
        spec.ilines = f.ilines
        spec.xlines = f.xlines
        spec.offsets = offsts

        # Initializing the merged one
        with segyio.create(merged_file_path, spec) as s:
        
            # Index for the merge headers and trace
            merge_index = 0
            # Index for the original file's headers and trace
            stack_index = 0
            # For loop to set parameters according to the seismic lines and offset
            for il in spec.ilines:
                for xl in spec.xlines:
                    for offset in spec.offsets:
                        # Assigning headers [byte]
                        s.header[merge_index] = {segyio.su.tracl  : f.header[stack_index][1],
                                                 segyio.su.tracr  : f.header[stack_index][5],
                                                 segyio.su.fldr   : f.header[stack_index][9],
                                                 segyio.su.cdp    : f.header[stack_index][21],
                                                 segyio.su.cdpt   : f.header[stack_index][25],
                                                 segyio.su.offset : offset, # 37
                                                 segyio.su.scalco : f.header[stack_index][71],
                                                 segyio.su.ns     : f.header[stack_index][115],
                                                 segyio.su.dt     : f.header[stack_index][117],
                                                 segyio.su.cdpx   : f.header[stack_index][181],
                                                 segyio.su.cdpy   : f.header[stack_index][185],
                                                 segyio.su.iline  : il,  # 189
                                                 segyio.su.xline  : xl}  # 193
                        if offset == list_of_angles[0]:
                            stack = f # near
                        if offset == list_of_angles[1]:
                            stack = g # mid
                        if offset == list_of_angles[2]:
                            stack = h # far                   
                        
                        # Copying the amplitudes from each stack
                        s.trace[merge_index] = stack.trace[stack_index]
                        
                        merge_index += 1
                    stack_index += 1
                
    return (f"Successful merge. New SEG-Y file path: {merged_file_path}")

In [15]:
merging_stacks(gather_path, merged_stacks, angle_list)

'Successful merge. New SEG-Y file path: ../data/NMF2950-2180_3000-2230.sgy'

# Making the SEG-Y files to store the AVO attributes

In [16]:
def avo_storing_files(list_of_gathers, intercept_path, gradient_path): 
    # Initializing the stacks
    with segyio.open(list_of_gathers[0]) as f:
        
        # Spec function to build the new segy
        spec = segyio.tools.metadata(f)

        # Initializing the merged one
        with segyio.create(gradient_path, spec) as gradient, segyio.create(intercept_path, spec) as intercept:

            # Assigning headers [byte]
            gradient.text[0], intercept.text[0] = f.text[0], f.text[0]
            gradient.bin, intercept.bin = f.bin, f.bin
            gradient.header, intercept.header = f.header, f.header
            gradient.trace, intercept.trace = f.trace, f.trace
            gradient.trace[0], intercept.trace[0] = np.zeros([1501,]), np.zeros([1501,])

                
    return (f"Successful construction. (Gradient, Intercept) path: ({intercept_path}, {gradient_path})")

In [17]:
avo_storing_files(gather_path, intercept_segy, gradient_segy)

/home/lsd/anaconda3/envs/tesis/lib/python3.7/site-packages/segyio/utils.py:23: RuntimeWarning: Implicit conversion from float64 to float32 (narrowing)
  warnings.warn(msg.format(x.dtype, dtype), RuntimeWarning)


'Successful construction. (Gradient, Intercept) path: (../data/intercept.sgy, ../data/gradient.sgy)'

# Function to give arguments to the pool

In [18]:
def input_generator(gather_file, list_of_angles, intercept_file, gradient_file):
    """
    
    """
    index = 0
    with segyio.open(gather_file, "r") as gather:
        for iline_number in gather.ilines:
            for xline_number in gather.xlines:
                    yield [gather_file, list_of_angles, iline_number, xline_number, index, intercept_file, gradient_file]
                    index += 1
                    
# test it
#input_args = input_generator(merged_stacks, angle_list, intercept_segy, gradient_segy)
#
#for row in input_args:
#    print(row)

# Function to compute the AVO attributes by trace

In [19]:
from scipy import stats

In [20]:
from scipy import stats
def  AVO_attributes_computation(gather_file, iline_number, xline_number, list_of_angles):
    """
    
    """
    sins = []
    for angle in angle_list:
        sins += [np.sin(np.radians(angle)) * np.sin(np.radians(angle))]
    
    with segyio.open(gather_file, "r") as gather:
        for time_slice in range(len(gather.samples)):
            amp = []
            for angle in list_of_angles:
                amp += [gather.gather[iline_number, xline_number, angle][time_slice]]
            
            # Regression
            gradient, intercept, r_value, p_value, std_error = stats.linregress(sins, amp)
            
            yield[intercept, gradient]

# Test it
#avo_attributes = AVO_attributes_computation(merged_stacks, 2950, 2180, angle_list)
#for row in avo_attributes:
#    print(row)

# Function to store the AVO attributes by trace

In [21]:
def store_AVO_attributes(generator_list):
    """
    
    """
    
    # Parameters to use in the functions
    gather_file = generator_list[0]
    list_of_angles = generator_list[1]
    iline_number = generator_list[2]
    xline_number = generator_list[3]
    index = generator_list[4]
    intercept_file = generator_list[5]
    gradient_file = generator_list[6]
    
    to_store = AVO_attributes_computation(gather_file, iline_number, xline_number, 
                                          list_of_angles)
    cont = 0
    with segyio.open(intercept_file, "r+") as intercept, segyio.open(gradient_file, "r+") as gradient:
        intercept_array = np.zeros([len(intercept.samples), ])
        gradient_array = np.zeros([len(gradient.samples), ])
        for attribute in to_store:
            intercept_array[cont] = attribute[0]
            gradient_array[cont] = attribute[1]
            cont += 1
        
        # Storing
        intercept.trace[index] = intercept_array
        gradient.trace[index] = gradient_array
        
        print(f"AVO attributes for trace [{iline_number},{xline_number}] has been stored successfully")
        return()
    
#input_args = input_generator(merged_stacks, angle_list, intercept_segy, gradient_segy)
#
#for row in input_args:
#    print(store_AVO_attributes(row))

# Integrating the previous functions [MP]

In [22]:
from multiprocessing import Pool

In [23]:
def avo_storeputation(gather_file, list_of_angles, intercept_file, gradient_file):
    
    input_data = input_generator(gather_file, list_of_angles, intercept_file, gradient_file)
    
    if __name__ == "__main__":
        p = Pool()
        p.map(store_AVO_attributes, input_data)

In [24]:
avo_storeputation(merged_stacks, angle_list, intercept_segy, gradient_segy)

/home/lsd/anaconda3/envs/tesis/lib/python3.7/site-packages/segyio/utils.py:23: RuntimeWarning: Implicit conversion from float64 to float32 (narrowing)
  warnings.warn(msg.format(x.dtype, dtype), RuntimeWarning)


AVO attributes for trace [2956,2200] has been stored successfully


/home/lsd/anaconda3/envs/tesis/lib/python3.7/site-packages/segyio/utils.py:23: RuntimeWarning: Implicit conversion from float64 to float32 (narrowing)
  warnings.warn(msg.format(x.dtype, dtype), RuntimeWarning)
/home/lsd/anaconda3/envs/tesis/lib/python3.7/site-packages/segyio/utils.py:23: RuntimeWarning: Implicit conversion from float64 to float32 (narrowing)
  warnings.warn(msg.format(x.dtype, dtype), RuntimeWarning)


AVO attributes for trace [2950,2180] has been stored successfully
AVO attributes for trace [2959,2210] has been stored successfully


/home/lsd/anaconda3/envs/tesis/lib/python3.7/site-packages/segyio/utils.py:23: RuntimeWarning: Implicit conversion from float64 to float32 (narrowing)
  warnings.warn(msg.format(x.dtype, dtype), RuntimeWarning)


...

AVO attributes for trace [2998,2187] has been stored successfully
AVO attributes for trace [2994,2229] has been stored successfully
AVO attributes for trace [2991,2221] has been stored successfully
AVO attributes for trace [2988,2211] has been stored successfully
AVO attributes for trace [2998,2188] has been stored successfully
AVO attributes for trace [2994,2230] has been stored successfully
AVO attributes for trace [2991,2222] has been stored successfully
AVO attributes for trace [2988,2212] has been stored successfully
AVO attributes for trace [2998,2189] has been stored successfully
AVO attributes for trace [2995,2180] has been stored successfully
AVO attributes for trace [2988,2213] has been stored successfully
AVO attributes for trace [2991,2223] has been stored successfully
AVO attributes for trace [2998,2190] has been stored successfully
AVO attributes for trace [2995,2181] has been stored successfully
AVO attributes for trace [2988,2214] has been stored successfully
AVO attrib

AVO attributes for trace [2989,2191] has been stored successfully
AVO attributes for trace [2995,2209] has been stored successfully
AVO attributes for trace [2992,2201] has been stored successfully
AVO attributes for trace [2998,2219] has been stored successfully
AVO attributes for trace [2989,2192] has been stored successfully
AVO attributes for trace [2995,2210] has been stored successfully
AVO attributes for trace [2992,2202] has been stored successfully
AVO attributes for trace [2998,2220] has been stored successfully
AVO attributes for trace [2989,2193] has been stored successfully
AVO attributes for trace [2995,2211] has been stored successfully
AVO attributes for trace [2992,2203] has been stored successfully
AVO attributes for trace [2998,2221] has been stored successfully
AVO attributes for trace [2989,2194] has been stored successfully
AVO attributes for trace [2995,2212] has been stored successfully
AVO attributes for trace [2992,2204] has been stored successfully
AVO attrib

AVO attributes for trace [2993,2181] has been stored successfully
AVO attributes for trace [2999,2199] has been stored successfully
AVO attributes for trace [2996,2189] has been stored successfully
AVO attributes for trace [2989,2223] has been stored successfully
AVO attributes for trace [2993,2182] has been stored successfully
AVO attributes for trace [2999,2200] has been stored successfully
AVO attributes for trace [2996,2190] has been stored successfully
AVO attributes for trace [2989,2224] has been stored successfully
AVO attributes for trace [2993,2183] has been stored successfully
AVO attributes for trace [2999,2201] has been stored successfully
AVO attributes for trace [2996,2191] has been stored successfully
AVO attributes for trace [2989,2225] has been stored successfully
AVO attributes for trace [2999,2202] has been stored successfully
AVO attributes for trace [2993,2184] has been stored successfully
AVO attributes for trace [2996,2192] has been stored successfully
AVO attrib

AVO attributes for trace [2996,2220] has been stored successfully
AVO attributes for trace [2990,2202] has been stored successfully
AVO attributes for trace [2993,2213] has been stored successfully
AVO attributes for trace [3000,2180] has been stored successfully
AVO attributes for trace [2996,2221] has been stored successfully
AVO attributes for trace [2990,2203] has been stored successfully
AVO attributes for trace [2993,2214] has been stored successfully
AVO attributes for trace [3000,2181] has been stored successfully
AVO attributes for trace [2996,2222] has been stored successfully
AVO attributes for trace [2990,2204] has been stored successfully
AVO attributes for trace [2993,2215] has been stored successfully
AVO attributes for trace [3000,2182] has been stored successfully
AVO attributes for trace [2996,2223] has been stored successfully
AVO attributes for trace [2990,2205] has been stored successfully
AVO attributes for trace [2993,2216] has been stored successfully
AVO attrib

AVO attributes for trace [3000,2210] has been stored successfully
AVO attributes for trace [2997,2201] has been stored successfully
AVO attributes for trace [2994,2193] has been stored successfully
AVO attributes for trace [2991,2183] has been stored successfully
AVO attributes for trace [3000,2211] has been stored successfully
AVO attributes for trace [2997,2202] has been stored successfully
AVO attributes for trace [2991,2184] has been stored successfully
AVO attributes for trace [2994,2194] has been stored successfully
AVO attributes for trace [3000,2212] has been stored successfully
AVO attributes for trace [2997,2203] has been stored successfully
AVO attributes for trace [2991,2185] has been stored successfully
AVO attributes for trace [2994,2195] has been stored successfully
AVO attributes for trace [3000,2213] has been stored successfully
AVO attributes for trace [2997,2204] has been stored successfully
AVO attributes for trace [2991,2186] has been stored successfully
AVO attrib